# Preprocessing the data
## Attempt #1 at increasing the accuracy rate to above 75%
### Removing noisy variables from features by removing an extra column and reducing classification value counts

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the EIN AND NAME columns 
application_df= application_df.drop(columns=["EIN","NAME","SPECIAL_CONSIDERATIONS"])

In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df['APPLICATION_TYPE'].value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = application_type[application_type < 500].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = classification_counts[classification_counts < 1500].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes=="object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Kaylaa\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True).drop(columns = application_cat)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

C:\Users\Kaylaa\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 80
hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=input_number, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3360      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [15]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 836us/step - loss: 62625.5625 - accuracy: 0.4867
Epoch 2/100
804/804 [==============================] - 1s 919us/step - loss: 66731.3281 - accuracy: 0.4890
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 44064.1680 - accuracy: 0.4896
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 20724.9590 - accuracy: 0.5045
Epoch 5/100
777/804 [===========================>..] - ETA: 0s - loss: 19154.5117 - accuracy: 0.5020
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 18583.7402 - accuracy: 0.5006
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 21951.3184 - accuracy: 0.5033
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 15235.8037 - accuracy: 0.5212
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 23661.4668 - accuracy: 0.4895
Epoch 9/100
80

804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 65/100
526/804 [==================>...........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5314
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 70/100
497/804 [=================>............] - ETA: 0s - loss: 0.6909 - accuracy: 0.5336
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 71/100
804/804 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7635 - accuracy: 0.5355 - 358ms/epoch - 1ms/step
Loss: 0.7635406255722046, Accuracy: 0.5355101823806763


## Attempt #2
### Adding an additional hidden layer and additional neurons to the hidden layers

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 100
hidden_layer2 = 50
hidden_layer3 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=input_number, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4200      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 4)                 204       
                                                                 
 dense_6 (Dense)             (None, 1)                 5         
                                                                 
Total params: 9,459
Trainable params: 9,459
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the weights
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [19]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 794.5460 - accuracy: 0.5292
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 5/100
781/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5323
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 900us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 65/100
496/804 [=================>............] - ETA: 0s - loss: 0.6919 - accuracy: 0.5253
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 980us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 70/100
487/804 [=================>............] - ETA: 0s - loss: 0.6914 - accuracy: 0.5298
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 71/100
804/

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6860 - accuracy: 0.5385 - 312ms/epoch - 1ms/step
Loss: 0.6859802007675171, Accuracy: 0.5385422706604004


# Attempt #3
### Changing the activation function of the hidden layers

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_number = len(X_train[0])
hidden_layer1 = 100
hidden_layer2 = 50
hidden_layer3 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=input_number, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               4200      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 4)                 204       
                                                                 
 dense_10 (Dense)            (None, 1)                 5         
                                                                 
Total params: 9,459
Trainable params: 9,459
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the weights
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [23]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6928 - accuracy: 0.5266
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5318
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 5/100
760/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5310
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 65/100
511/804 [==================>...........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5318
Epoch 69/100
804/804 [==============================] - 1s 930us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 70/100
486/804 [=================>............] - ETA: 0s - loss: 0.6916 - accuracy: 0.5287
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 71/100
804/80

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7235 - accuracy: 0.4282 - 468ms/epoch - 2ms/step
Loss: 0.7234799265861511, Accuracy: 0.42822158336639404


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")